## **Mount Google Drive to Store Files and Data**

In [ ]:
# Berikut adalah link yang dapat Anda set dalam kode program Anda. Anda dapat mendownload dataset yang Anda butuhkan melalui link berikut:
# Link 1000 images: https://www.dropbox.com/s/c0yfe4bpt0orb0m/paintings1k.zip?dl=1
# Link 5000 images: https://www.dropbox.com/s/yhu8v2yhft70doh/paintings5k.zip?dl=1
# Link 10000 images: https://www.dropbox.com/s/16jrkfrdm0c543q/paintings15k.zip?dl=1

# File yang terdownload akan dalam format .zip. Silakan disimpan di direktori Anda.

In [ ]:
# Gunakan fungsi ini jika Anda menjalankan kode program pada google collab dan menggunakan google drive untuk menyimpan file.

from google.colab import drive
drive.mount('/content/drive')

# Jika Anda menjalankan pada komputer Anda, abaikan blok program ini.

## **1) Importing Python Packages for GAN**


In [ ]:
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D, Dense, Conv2DTranspose
from keras.layers import Dropout
from keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
!mkdir generated_images resized_images

In [ ]:
# Unzip the-zip-file -d name-of-destination-folder
# the-zip-file: path dari folder sumber data yang akan Anda gunakan.
# name-of-destination-folder: nama folder yang tempat hasil file yang akan di unzip disimpan.
# Contoh: !unzip /content/drive/MyDrive/paintings1k.zip -d 5kImage

In [ ]:
!unzip /content/drive/MyDrive/paintings1k.zip -d 5kImage

In [ ]:
# Isi bagian didalam tanda "" dengan alamat path dari image yang akan Anda gunakan.
# Hint: klik kanan pada folder yang Anda tuju, kemudian pilih copy-path.
# Contoh: /content/drive/5kImage

images_path = "<...>"

## **Resizing Data to match Neural Network Input**

In [ ]:
import os
# from PIL import Image
import cv2
reshape_size = (64,64)

i = 0
for image in os.listdir(images_path):
  # print(image)
  img = cv2.imread(images_path + image)
  img = cv2.resize(img, reshape_size)
  cv2.imwrite("resized_images/%d.png" % i,img)
  # print(img.shape)
  i = i+1

## **2) Parameters for Neural Networks & Data**

In [ ]:
img_width = 64
img_height = 64
channels = 3
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(learning_rate=0.0002)

## **3) Building Generator**





In [ ]:
def build_generator():
    model = Sequential()
    model.add(Dense(256 * 8* 8, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((8,8,256)))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))

    model.summary()

    return model

generator = build_generator()

## **4) Building Discriminator**

In [ ]:
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, (3,3), padding='same', input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (3,3), padding='same', ))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(256, (3,3), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    model.summary()
    return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

## **5) Connecting Neural Networks to build GAN**

In [ ]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
generator.summary()

In [ ]:
discriminator.summary()

## **6) Outputting Images**


In [ ]:
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 4, 4
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001

    # Rescale images 0 - 1
    gen_imgs = (gen_imgs + 1) / 2.0

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("currentgeneration.png")
    fig.savefig("generated_images/%.8f.png" % save_name)
    plt.close()

## **7) Training GAN**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

latent_dim = 100  # Define latent dimension

def train(epochs, batch_size=32, save_interval=200):
    (X_train, _), (_, _) = tf.keras.datasets.cifar10.load_data()
    X_train = X_train / 127.5 - 1.0  # Normalize to [-1, 1]

    bat_per_epo = int(X_train.shape[0] / batch_size)
    valid = np.ones((batch_size, 1))
    fakes = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        for j in range(bat_per_epo):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            gen_imgs = generator.predict(noise)

            # Train discriminator
            d_loss_real = discriminator.train_on_batch(imgs, valid)
            d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = GAN.train_on_batch(noise, valid)

            print("Epoch: %d Batch: %d [D loss: %f, acc: %.2f%%] [G loss: %f]" %
                  (epoch, j, d_loss[0], 100 * d_loss[1], g_loss))

        if epoch % save_interval == 0:
            generator.save(f"generator_epoch_{epoch}.h5")  # Save model at intervals

train(30000, batch_size=32, save_interval=200)

In [ ]:
noise = np.random.normal(0, 1, (16, latent_dim))
gen_imgs = generator.predict(noise)
gen_imgs = (gen_imgs + 1) / 2.0
# plt.imshow(gen_imgs[2])

In [ ]:
plt.imshow(gen_imgs[6])

In [ ]:
generator.save_weights("/content/drive/MyDrive/models/generator1hour.h5")
discriminator.save_weights("/content/drive/MyDrive/models/discriminator1hour.h5")

### **8) Making GIF**

In [ ]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)

  if filenames:  # Check if filenames is not empty
    image = imageio.imread(filenames[-1])
    writer.append_data(image)